In [ ]:
from colossus.cosmology import cosmology
from colossus.lss import mass_function
import matplotlib.pyplot as plt
import numpy as np
import math
from dataloc import HMF_T08_BOL_FILE

x_mass = np.logspace(8.0, 16, 100)

plt.rcParams.update({'font.size': 16})
plt.figure(figsize=(10,6))
cosmology.setCosmology('bolshoi')

# Let's compare to whats in the halo mass function file we are using
fn = HMF_T08_BOL_FILE
data = np.loadtxt(fn)
plt.plot(data[:,0], data[:,0]*data[:,1], 'r--', label='GF Default')

for z in [0.0]:
#for z in [0.0, 0.33, 0.75, 1.25, 2.2, 3.2, 5.5, 10.0]:
#for z in [0.0, 0.1, 0.2]:
    age = cosmology.getCurrent().age(z)
    bf = 0.156
    mfunc = mass_function.massFunction(x_mass, z, mdef = '200m', model = 'tinker08', q_out='dndlnM') 
    #color = plt.cm.inferno(0.9 - (age / 13.8))
    color = plt.cm.inferno(z / 0.3)
    plt.plot(x_mass, mfunc, color=color, label=f'z={z:.3f} ({age:.1f} Gyr)')
    
plt.xscale('log')
plt.yscale('log')
#plt.ylim(1e-3, 1e0)
plt.xlabel('$M_h$ [$M_\odot$] (Bolshoi Cosmology)')
plt.ylabel('Number Density')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()

In [ ]:
# Let's try to reproduce the existing file.
# The first cell uses 'bolshoi' cosmology and 'tinker08' model.
# The output of massFunction with q_out='dndlnM' gives dn/dlnM.
# We need dn/dM. The conversion is dn/dM = (dn/dlnM) / M.

cosmology.setCosmology('bolshoi')
# Using the same mass bins as the file appears to have from the plot.
mass_bins = data[:,0]
mfunc_dndlnM = mass_function.massFunction(mass_bins, 0.0, mdef = '200m', model = 'tinker08', q_out='dndlnM')
mfunc_dndM = mfunc_dndlnM / mass_bins

# Save to a temporary file
temp_filename = 'halo_mass_function_bolshoi.dat'
np.savetxt(temp_filename, np.c_[mass_bins, mfunc_dndM], fmt='%e')

print(f"File '{temp_filename}' created.")

In [ ]:
# Now, let's compare the original file with our new one.
original_data = np.loadtxt(HMF_T08_BOL_FILE)
new_data_bolshoi = np.loadtxt(temp_filename)

# Check if they are close, only comparing the first two columns
if np.allclose(original_data[:,:2], new_data_bolshoi):
    print("The newly generated file is very close to the original file.")
else:
    print("The files are different. Let's check the difference.")
# Plot the fractional difference
plt.figure(figsize=(10,6))
plt.plot(original_data[:,0], (new_data_bolshoi[:,1] - original_data[:,1]) / original_data[:,1])
plt.xscale('log')
plt.xlabel('$M_h$ [$M_\odot$]')
plt.ylabel('Fractional Difference')
plt.title('Difference between original and regenerated Bolshoi HMF')
plt.grid(True)
plt.show()

In [ ]:
# Now create the new halo mass function file with Planck18 cosmology.
cosmology.setCosmology('planck18')
mfunc_planck_dndlnM = mass_function.massFunction(mass_bins, 0.0, mdef = '200m', model = 'tinker08', q_out='dndlnM')
mfunc_planck_dndM = mfunc_planck_dndlnM / mass_bins

# Save to the new file
new_filename = '../halo_mass_function_planck18.dat'
np.savetxt(new_filename, np.c_[mass_bins, mfunc_planck_dndM], fmt='%e')

print(f"New halo mass function file '{new_filename}' created with Planck18 cosmology.")

# Let's plot the new one against the old one
plt.figure(figsize=(10,6))
plt.plot(data[:,0], data[:,0]*data[:,1], 'r--', label='GF Default (Bolshoi)')
plt.plot(mass_bins, mass_bins*mfunc_planck_dndM, 'b-', label='New (Planck18)')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$M_h$ [$M_\odot$]')
plt.ylabel('$M_h \cdot dN/dM$')
plt.title('Comparison of Halo Mass Functions')
plt.legend()
plt.grid(True)
plt.show()

# How close is it?
if np.allclose(original_data[:,:2], np.c_[mass_bins, mfunc_planck_dndM]):
    print("The Planck18 HMF is very close to the original Bolshoi HMF.")
else:
    print("The Planck18 HMF differs from the original Bolshoi HMF.")
    # Plot the fractional difference
    plt.figure(figsize=(10,6))
    fractional_difference = (mfunc_planck_dndM - original_data[:,1]) / original_data[:,1]
    plt.plot(mass_bins, fractional_difference, 'k-')
    plt.xscale('log')
    plt.xlabel('$M_h$ [$M_\odot$]')
    plt.ylabel('Fractional Difference')
    plt.title('Fractional Difference between Planck18 and Bolshoi HMFs')
    plt.grid(True)
    plt.show()